In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time
import os
import sys
sys.path.append('../../')

from data.processing import get_data

import models

from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new
from main.seir.sensitivity import calculate_sensitivity_and_plot
from utils.generic.create_report import save_dict_and_create_report
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
#from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles

import yaml

In [ ]:
from os.path import exists, join, splitext

In [ ]:
config_filename = 'uncer.yaml'
config1 = read_config(config_filename)
config2 = read_config('default.yaml')

In [ ]:
runs = 1
mcmc_P = []
bo_P = []
mcmc_perc_loss = []
bo_perc_loss = []
k = []
PD = {}
for n in range(runs):
    PD[f'm{n}']= {}
    percentile_dict1 = {}
    percentile_dict2 = {}
    percentile_dict1['m1'] = single_fitting_cycle(**copy.deepcopy(config1['fitting']))
    percentile_dict2['m1'] = single_fitting_cycle(**copy.deepcopy(config2['fitting']))
    percentile_dict1['m1']['forecasts'] = {}
    percentile_dict1['m1']['forecasts']['best'] = get_forecast(percentile_dict1, train_fit='m1', 
                                                            model=config1['fitting']['model'], 
                                                            forecast_days=config1['forecast']['forecast_days'])

    percentile_dict1['m1']['trials_processed'] = forecast_all_trials(percentile_dict1, train_fit='m1', 
                                                                    model=config1['fitting']['model'], 
                                                                    forecast_days=config1['forecast']['forecast_days'])
    percentile_dict2['m1']['forecasts'] = {}
    percentile_dict2['m1']['forecasts']['best'] = get_forecast(percentile_dict2, train_fit='m1', 
                                                            model=config2['fitting']['model'], 
                                                            forecast_days=config2['forecast']['forecast_days'])

    percentile_dict2['m1']['trials_processed'] = forecast_all_trials(percentile_dict2, train_fit='m1', 
                                                                    model=config2['fitting']['model'], 
                                                                    forecast_days=config2['forecast']['forecast_days'])

    uncertainty_args1 = {'predictions_dict': percentile_dict1, **config1['uncertainty']['uncertainty_params']}
    uncertainty1 = config1['uncertainty']['method'](**uncertainty_args1)
    uncertainty_args2 = {'predictions_dict': percentile_dict2, 'fitting_config': config2['fitting'],
                    'forecast_config': config2['forecast'], **config2['uncertainty']['uncertainty_params']}
    uncertainty2 = config2['uncertainty']['method'](**uncertainty_args2)
    uncertainty_forecasts1 = uncertainty1.get_forecasts()
    uncertainty_forecasts2 = uncertainty2.get_forecasts()
    mcmc = []
    bo = []
    key = []
    for i in uncertainty_forecasts1.keys():
        key.append(i)
        mcmc.append(uncertainty_forecasts1[i]['df_loss_perc'].mean()[0].item())
        bo.append(uncertainty_forecasts2[i]['df_loss_perc'].mean()[0])
    k = key
    mcmc_perc_loss.append(mcmc)
    bo_perc_loss.append(bo)
    mcmc_P.append(uncertainty1.p_val)
    bo_P.append(uncertainty2.p_val)
    PD[f'm{n}']['MCMC'] = percentile_dict1
    PD[f'm{n}']['BO'] = percentile_dict2
    
    

In [ ]:
m = np.array(mcmc_perc_loss)
b = np.array(bo_perc_loss)


In [ ]:
m = np.mean(m,axis = 0)
b = np.mean(b,axis = 0)

In [ ]:
PD['perc_loss'] = {}
PD['perc_loss']['mcmc'] = m
PD['perc_loss']['bo'] = b

In [ ]:
import pickle as pkl
with open('../../misc/predictions/predictions_dict_perc_20.pickle', 'wb') as handle:
    pkl.dump(PD, handle)


In [ ]:
with open('../../misc/predictions/predictions_dict_perc_20.pickle', 'rb') as handle:
    PD = pkl.load(handle)

In [ ]:
plt.figure(figsize= (15,7))
mc, = plt.plot(key,m,markersize=12,label='MCMC',color='b')
bo, = plt.plot(key,b,markersize=12,label = 'BO',color='r')
plt.legend(handles = [mc,bo])
plt.xlabel('Percentiles')
plt.ylabel('Percentile Loss in Total compartment')

In [ ]:
m.mean()

In [ ]:
b.mean()

In [ ]:
import numpy as np
np.linspace(1,99,40)